In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io.fits import getdata

import pandas as pd
numaster = getdata('numaster.fits')

from astropy.time import Time

cutoff_time = Time('2021-12-31T00:00:00').mjd



In [2]:
df = pd.read_csv('stray_light_det_v2.txt', sep='\s+', names=['SEQID', 'FPM', 'SRC', 'DET0', 'DET1', 'DET2', 'DET3'])

# Filter out cases where the source is really bright:
bright_lim = 100.
df = df[df['SRC']<100.]


df['DET0123'] = df['DET0'] + df['DET1'] + df['DET2'] + df['DET3']
df['DET01'] = df['DET0'] + df['DET1']
df['DET12'] = df['DET1'] + df['DET2']
df['DET23'] = df['DET2'] + df['DET3']
df['DET03'] = df['DET0'] + df['DET3']

names = []
exp = []
ra = []
dec = []
pa = []
mjds = []
for row in df['SEQID']:
    nuind = np.where(numaster['OBSID'] == f'{row}')

    if (len(nuind[0]) == 0):
        names = np.append(names, '')
        exp = np.append(exp, 0)
        ra = np.append(ra, 0.)
        dec = np.append(dec, 0.)
        pa = np.append(pa, 0.)
        mjds = np.append(mjds, 0.)
    else:
        names = np.append(names, numaster['NAME'][nuind[0]].strip())
        exp = np.append(exp, numaster['EXPOSURE_A'][nuind[0]])
        ra = np.append(ra,numaster['RA'][nuind[0]] )
        dec = np.append(dec,numaster['DEC'][nuind[0]] )
        pa = np.append(pa,numaster['ROLL_ANGLE'][nuind[0]] )
        mjds = np.append(mjds,numaster['TIME'][nuind[0]] )


        
df['NAME'] = names
df['EXPOSURE'] = exp
df['RA'] = ra
df['DEC'] = dec
df['PA'] = pa
df['TIME'] = mjds

# Get rid of stuff with low exposure
df = df[(df['EXPOSURE']>1e3) & (df['TIME']<cutoff_time)]


# Get rid of stuff near SgrA*, GalSurveys, Gal_Cen, or the GC Magnetar:
# Also Coma and Bullet cluster stuff
bad_name = ['Sgr', 'GalSurvey', 'Gal_Cen', 'gcmag', 'Coma', 'bullet', 'Bullet']
for bn in bad_name:   
    df = df[~(df['NAME'].str.startswith(bn))]

In [3]:
sig_list = []

merged_df = pd.read_csv('straycats2_merged.csv')



sig_list = merged_df['SEQID'].values

# with open('straycats2_merged.csv', 'r') as f:
#     for line in f:
#         fields = line.split(',')
# #         if fields[0].strip() == '':
# #             break
#         if fields[1] not in sig_list:
#             sig_list = np.append(sig_list, fields[1])
# sig_list = sig_list[1:]

In [4]:
print(len(df))
print(len(sig_list))

for seq in sig_list:
    try:
        seqid = float(seq)
    except:
        continue
    df = df[~(df['SEQID'] == seqid)].copy().reset_index(drop=True)
print(len(df))


877
106
777


In [5]:
# Filter out bright sources and short exposures
df = df[(df['SRC'] < 2.) & (df['EXPOSURE'] > 10e3)].copy().reset_index(drop=True)
print(len(df))

677


In [6]:
dfa = df[df['FPM']=='A'].copy().reset_index(drop=True)
dfb = df[df['FPM']=='B'].copy().reset_index(drop=True)

running_list = {'A':[], 'B':[]}


# Iteration step: find stuff 3-sigma

sig_limit = 5.0
for det_comb in ['DET0123', 'DET12', 'DET23', 'DET01', 'DET03', 'DET1', 'DET2', 'DET3', 'DET0']:
    print(det_comb)
    print()
    
    use = {'A':np.full(len(dfa), True, dtype=bool),
           'B':np.full(len(dfb), True, dtype=bool)}
    for fpm, dfi in zip(['A', 'B'], [dfa, dfb]):
        with open(f'sig_cuts_{det_comb}{fpm}_lvl2.txt', 'w') as f:
            for iter in range(3):
                dfi['SIG'] = (dfi[det_comb] - dfi[det_comb].mean()) / dfi[det_comb].std()
                high_sig = dfi[dfi['SIG'] > sig_limit]

                for ind, row in high_sig.iterrows():
                    use[fpm][ind] = False
                    if row['SEQID'] not in running_list[fpm]:
                        running_list[fpm] = np.append(running_list[fpm], row['SEQID'])
                        outstring = f"{row['SEQID']}, {row['NAME']}, {row['EXPOSURE']}, {row['RA']}, {row['DEC']}, {row['PA']}, {iter}"
                        print(fpm, row['SRC'], row['EXPOSURE'], row['SIG'], outstring)
                        f.write(outstring+'\n')
                    dfi = dfi[dfi['SIG']<sig_limit].copy()

    # Now get rid of everything that you've already used for the next round of searching.
    dfa = dfa[use['A']].copy().reset_index(drop=True)
    dfb = dfb[use['B']].copy().reset_index(drop=True)

DET0123

A 0.5320148773057951 47964.8241 6.026996488856269 60602021004, M87, 47964.8241, 187.6611, 12.3865, 28.7368, 0
A 0.529361188133461 48770.1036 6.07658727081034 60602021002, M87, 48770.1036, 187.6612, 12.3905, 32.7813, 0
A 0.1086036524758922 107114.2612 8.016931870751574 30601014003, 2FHL_J1745d1m3035, 107114.2612, 266.3299, -30.5272, 158.573, 0
A 0.1568186801043871 106116.1846 7.043178686190087 80601502001, Circinus_ULX5, 106116.1846, 213.1882, -65.3275, 102.6844, 0
A 1.187063119708284 101749.4335 6.407042925158518 70660001002, ABELL_2029, 101749.4335, 227.7529, 5.7465, 164.7726, 1
A 0.6533700752504995 52526.1277 6.043759683215688 70660004004, ABELL_2199, 52526.1277, 247.1419, 39.5392, 320.0378, 1
A 0.656178983165541 85603.1684 5.985578778533496 70660004002, ABELL_2199, 85603.1684, 247.1378, 39.5406, 320.0401, 1
A 0.7556124896055275 52861.752 5.286336284812256 70660003004, ABELL_1795, 52861.752, 207.1976, 26.5853, 346.181, 1
A 1.6172604266342168 205271.2071 6.265765349913181 306

In [86]:
print(len(running_list['A']))
print(len(running_list['B']))



389
369
